In [22]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [23]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for scraping
url = "https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24"

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [63]:
# Collect table header
table = soup.find("table", class_="Crom_table__p1iZz")
headers = table.thead.find_all('th')

header = []
for i in range(len(headers)):
    header.append(headers[i].text)

['#',
 'Player',
 'TEAM',
 'GP',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3PM',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'EFF']

In [117]:
# Collect table data
for row in table.tbody.find_all('tr'):
    for i in range(len(row.find_all('td'))):
        row_data = row.find_all('td')[i]
        id = row_data[0]
        

KeyError: 0